In [ ]:
%load_ext autoreload


In [1]:
from random import random, randint
import numpy as np
from copy import deepcopy

In [8]:
a=2
b=3
2+3

5

In [7]:
etapa_dif = []
for i in range(1,32):
    print(i)
    etapa_dif.append(10*i)
personagem_agilidade = [1.8, 1.6, 1.6, 1.6, 1.4, 0.9, 0.7]


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [6]:
print(etapa_dif)

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310]


In [ ]:
import numpy as np
from random import random, randint

NUM_LINES = 30
NUM_COLUMNS = 7
ENERGY = 8
etapa_dif = []
for i in range(1,32):
    etapa_dif.append(10*i)
personagem_agilidade = [1.8, 1.6, 1.6, 1.6, 1.4, 0.9, 0.7]

def checa_vazio(m):    
    for i in range(len(m)):
        sum = m[i].sum()
        if sum == 0:
            return i
    return -1

def garante_vivo(m):
    if m.sum() < NUM_COLUMNS*ENERGY:
        return m
    # muda uma vida pois máximo possivel por construção é NUM_COLUMNS*ENERGY
    i = randint(0, NUM_LINES - 1)
    while m[i].sum() < 2:
        i = randint(0, NUM_LINES - 1)
    j = randint(0, NUM_COLUMNS - 1)
    while m[i][j] == 0:
        j = randint(0, NUM_COLUMNS - 1)
    m[i][j] = 0
    return m

# talvez usar garante_vivo
def cria_novo_matrix():
    m=np.zeros((NUM_LINES, NUM_COLUMNS))
    vivo = randint(0, NUM_COLUMNS-1)
    for i in range(NUM_COLUMNS):
      ltot = 0
      if i == vivo:
          ltot = 1 ## i irá sobreviver
      while ltot < ENERGY:
         n = randint(0, NUM_LINES - 1)
         if m[n][i] != 1:
               m[n][i] = 1
               ltot += 1
    index = checa_vazio(m)
    while index != -1:
        n = randint(0, NUM_COLUMNS - 1)
        m[index][n] = 1
        ## atualizar matrix pois agora tem mais de k 1's na coluna
        k = randint(0, NUM_LINES - 1)
        while m[k][n]== 0:
            k = randint(0, NUM_LINES - 1)
        m[k][n] = 0
        index = checa_vazio(m)
    return m 

def custo_tempo(etapa_dif, personagem_agilidade, matrix_genetica):
    matrix = np.divide(etapa_dif, np.matmul(matrix_genetica, personagem_agilidade))
    return np.sum(matrix) # returna tempo total de todas etapas

def busca(p, lista): # returna indice i em que lista[i-1] < p < lista[i]
	low = 0
	high = len(lista) - 1
	while low <= high:
		middle = low + (high - low) // 2
		if lista[middle] >= p and lista[middle-1] < p: # lista[middle-1] < p <= lista[middle]
			return middle
		elif lista[middle] <= p:
			low = middle + 1 
		else:
			high = middle - 1
	return 0

def random_selection(population):
    times = list()
    for person in population:
        times.append(person[1])
    times = np.divide(1, times) 
    probs = np.divide(times, np.sum(times)) ## mais provavel, menor tempos
    lista = []
    psum = 0
    n= len(probs)
    for i in range(n):
        psum += probs[i]
        lista.append(psum)
    p = random()
    j = busca(p, lista)
    return population[j] # (person, time)

def best_individual(population):
    population.sort(key=lambda y: y[1]) #sort by time
    best_time = population[0][1]
    best_person = population[0][0]
    return best_time, best_person

def die(population, max):
    tam = len(population)
    excess = tam - max
    # tupla (child, time)
    population.sort(key=lambda y: y[1])
    for i in range(1,excess+1):
        population.pop(tam-i)
    # return population

def repetition(population, potential_child):
    for child in population:
        if np.array_equal(child[0], potential_child[0]):
            return 1
    return 0

# mae  = matrix
def reproduce(mae, pai):
    m = np.zeros((NUM_LINES, NUM_COLUMNS))
    for i in range (NUM_COLUMNS):
        if randint(0,1):
           m[:, i]=mae[:, i]
        else:
            m[:, i]=pai[:, i]
    if checa_vazio(*m) != -1: 
        return reproduce(mae,pai)
    m = garante_vivo(m)
    return m


def genetic_algorithm(population): ## population list of matrices
    best_time, fit_ind = best_individual(population)
    print('best_time', best_time)
    count_change = 0
    while count_change < 50:
        new_population = []
        # evaluate best individual...
        for i in range(100):
            x = random_selection(population) # (person, time)
            y = random_selection(population)
            while np.array_equal(x[0],y[0]):
                y = random_selection(population)
            child = reproduce(x[0],y[0])
            tempo_child = custo_tempo(etapa_dif, personagem_agilidade, child)
            if repetition(new_population, child) == 0:
                new_population.append((child, tempo_child))
            # p = random()
            # if (p < 0.01):
            #     child = mutate(child)
        for person in population:
            new_population.append(person)
        population = new_population
        print('new pop', len(population))
        if len(population) > 2000:
            die(population, 2000)
            print('new pop after massive deaths', len(population))
        time, ind = best_individual(population)
        if time < best_time:
            best_time = time
            fit_ind = ind
            print('time', time)
            print('best_time', best_time)
        else:
            count_change += 1
            
    return fit_ind

In [ ]:
population = list()
count = 0
while count < 100:
    potential_person = cria_novo_matrix()
    if repetition(population, potential_person) == 0:
        tempo = custo_tempo(etapa_dif, personagem_agilidade, potential_person)
        population.append((potential_person, tempo))
        count +=1

In [ ]:
nlines = 31
ncols = 7
k = 8

def cria_linha(ncols): #ncols = 7
    l=[]
    i=0
    t=0
    while i<ncols:
        n=randint(0,1) # mudar probabilidade!!!!!!!! assim nao funciona
        l.append(n)
        t+=n
        i+=1
    if t:#checa se pelo menos um membro percorreu o percurso
        return l
    else:
        return cria_linha(ncols)
   
#cria matriz [30][7]
def cria_matriz(nlines, ncols): # nlines=30, ncols = 8
    i=0
    m=[]
    while i<nlines:
       m.append(cria_linha(ncols))
       i+=1
    return m


def cria_novo_matrix(nlines, ncols, k):
    m=np.zeros((nlines, ncols))
    vivo = randint(0, ncols-1)
    for i in range(ncols):
      ltot = 0
      if i == vivo:
          ltot = 1 ## i irá sobreviver
      while ltot < k:
         n = randint(0,nlines-1)
         if m[n][i] != 1:
               m[n][i] = 1
               ltot += 1
    index = checa_vazio(*m)
    while index != -1:
        n = randint(0,ncols-1)
        m[index][n] = 1
        ## atualizar matrix pois agora tem mais de k 1's na coluna
        k = randint(0, nlines-1)
        while m[k][n]== 0:
            k = randint(0, nlines-1)
        m[k][n] = 0
        index = checa_vazio(*m)
    return m 


In [ ]:
nlines = 31
ncols = 7
k = 8
#checa se alguem foi usado mais que k vezes
#checa se alguem foi usado mais que k vezes
def checa_k_vivo(*m):
    alive = 0
    for i in range(len(m[0])):
        t=0
        for j in range(len(m)):
            t+=(m[j][i])
            # j+=1z
        alive += t
        if t>k:
            return i #retorna indice do que apareceu mais de k vezes
    if alive >= k*len(m[0]):
        return -2
            
        # i+=1
    return -1


def checa_vazio(m):    
    for i in range(len(m)):
        sum = m[i].sum()
        if sum == 0:
            return i
    return -1
    
def bits(n):
    bits = [] 
    while n > 0:
        r = n%2
        n = n//2
        bits.append(r)
    return bits

# assume que soma resultado vai a no maximo n '1s
def soma_bits(a,b, n ):
    r = a+b
    for i in range(n-1):
        if r[i] == 2:
            r[i] = 0
            r[i+1] +=1
    return r

def reproduce1(x,y): 
    # x = (person, time)
    tuplas = []
    for line in range(nlines):
        for col in range(ncols):
            if x[0][line][col] != y[0][line][col]:
                tuplas.append((line, col))
    casas_sorteadas = []
    n = min(5, len(tuplas))
    for i in range(n):
        num = randint(0, len(tuplas)-1)
        casas_sorteadas.append(tuplas[num])
        tuplas.pop(num)
    childs = [x, y]
    bits = np.zeros(n)
    bits[0] = 1
    m = deepcopy(x[0])
    l = pow(2,n)
    for i in range(l-1): ## criancas podem ser x e y
        matrix = deepcopy(m)
        for j in range(n):
            # percorrer bits
            if bits[j] == 1:
                a = casas_sorteadas[j][0]
                b = casas_sorteadas[j][1]
                matrix[a][b] = y[0][a][b]
        um = np.zeros(n)
        um[0] = 1
        bits = soma_bits(bits, um, n)
        if checa_k_vivo(*matrix) == -1 and checa_vazio(*matrix) == -1:
            time = custo_tempo(etapa_dif, personagem_agilidade, matrix)
            childs.append((matrix, time))
    return childs

In [ ]:

etapa_dif = []
for i in range(1,32):
    etapa_dif.append(10*i)
personagem_agilidade = [1.8, 1.6, 1.6, 1.6, 1.4, 0.9, 0.7]
nlines = 31
ncols = 7
k = 8
# custo do tempo a cada etapa: dificuldade/ agilidade na etapa
def custo_tempo(etapa_dif, personagem_agilidade, matrix_genetica):
    matrix = np.divide(etapa_dif, np.matmul(matrix_genetica, personagem_agilidade))
    return np.sum(matrix)



def busca(p, lista): # returna indice i em que lista[i-1] < p < lista[i]
	low = 0
	high = len(lista) - 1
	while low <= high:
		middle = low + (high - low) // 2
		if lista[middle] >= p and lista[middle-1] < p: # lista[middle-1] < p <= lista[middle]
			return middle
		elif lista[middle] <= p:
			low = middle + 1 
		else:
			high = middle - 1
	return 0

def random_selection(population):
    times = list()
    for person in population:
        times.append(person[1])
    times = np.divide(1, times) 
    probs = np.divide(times, np.sum(times)) ## mais provavel, menor tempos
    lista = []
    psum = 0
    n= len(probs)
    for i in range(n):
        psum += probs[i]
        lista.append(psum)
    p = random()
    j = busca(p, lista)
    return population[j] # (person, time)
    
def best_individual(population):
    population.sort(key=lambda y: y[1]) #sort by time
    best_time = population[0][1]
    best_person = population[0][0]
    return best_time, best_person

def die(population, max):
    tam = len(population)
    excess = tam - max
    # tupla (child, time)
    population.sort(key=lambda y: y[1])
    for i in range(1,excess+1):
        population.pop(tam-i)
    # return population

def repetition(population, potential_child):
    for child in population:
        if np.array_equal(child[0], potential_child[0]):
            return 1
    return 0

def genetic_algorithm(population): ## population list of matrices
    best_time, fit_ind = best_individual(population)
    print('best_time', best_time)
    count = 0
    while count < 50:
    ## tamanho pop
    # ideia tirar caras ruins
        new_population = []
        # evaluate best individual...
        for i in range(200):
            x = random_selection(population) # (person, time)
            y = random_selection(population)
            while np.array_equal(x[0],y[0]):
                y = random_selection(population)
            childs = reproduce1(x,y)
            for child in childs:
                if repetition(new_population, child) == 0:
                    new_population.append(child)
            # p = random()
            # if (p < 0.01):
            #     child = mutate(child)
        population = new_population
        print('new pop', len(population))
        if len(population) > 3000:
            die(population, 3000)
            print('new pop after massive deaths', len(population))
        
        time, ind = best_individual(population)
        if time < best_time:
            best_time = time
            fit_ind = ind
            print('time', time)
            print('best_time', best_time)
        else:
            count += 1
            
    return fit_ind

In [ ]:
ENERGY = 2

In [ ]:
def reproduce(mae, pai):
    m = np.zeros((NUM_LINES, NUM_COLUMNS))
    for i in range (NUM_COLUMNS):
        if randint(0,1):
           m[:, i]=mae[:, i]
        else:
            m[:, i]=pai[:, i]
    if checa_vazio(m) != -1: 
        return reproduce(mae,pai)
    m = garante_vivo(m)
    return m

In [ ]:
def garante_vivo(m):
    if m.sum() < NUM_COLUMNS*ENERGY:
        return m
    # muda uma vida pois máximo possivel por construção é NUM_COLUMNS*ENERGY
    i = randint(0, NUM_LINES - 1)
    while m[i].sum() < 2:
        i = randint(0, NUM_LINES - 1)
    j = randint(0, NUM_COLUMNS - 1)
    while m[i][j] == 0:
        j = randint(0, NUM_COLUMNS - 1)
    m[i][j] = 0
    return m

In [ ]:
population = list()
count = 0
while count < 100:
    potential_person = cria_novo_matrix()
    if repetition(population, potential_person) == 0:
        tempo = custo_tempo(etapa_dif, personagem_agilidade, potential_person)
        population.append((potential_person, tempo))
        count +=1

In [ ]:
len(population)

In [ ]:

def cria_novo_matrix(k):
    m=np.zeros((NUM_LINES, NUM_COLUMNS))
    vivo = randint(0, NUM_COLUMNS-1)
    for i in range(NUM_COLUMNS):
      ltot = 0
      if i == vivo:
          ltot = 1 ## i irá sobreviver
      while ltot < k:
         n = randint(0, NUM_LINES - 1)
         if m[n][i] != 1:
               m[n][i] = 1
               ltot += 1
    index = checa_vazio(*m)
    while index != -1:
        n = randint(0, NUM_COLUMNS - 1)
        m[index][n] = 1
        ## atualizar matrix pois agora tem mais de k 1's na coluna
        k = randint(0, NUM_LINES - 1)
        while m[k][n]== 0:
            k = randint(0, NUM_LINES - 1)
        m[k][n] = 0
        index = checa_vazio(*m)
    return m 

In [ ]:
NUM_LINES = 4
NUM_COLUMNS = 3
cria_novo_matrix(2)

In [ ]:
lista = [1,2]
lista2 = [3,2]
lista2.append(lista)
lista2

In [ ]:
genetic_algorithm(population)

In [ ]:
time, ind = best_individual(population)

In [ ]:
time

In [ ]:
min(4,1)

In [ ]:
x = population[0]
y= population[1]

In [ ]:
x

In [ ]:
y

In [ ]:
childs= reproduce1(x,y,2)

In [ ]:
len(childs)

In [ ]:
np.array_equal(childs[1], childs[2])

In [ ]:
a = genetic_algorithm(population, 40)

In [ ]:
from random import random

In [ ]:
m = np.matrix([[0,1], [1,1], [0,0]])
checa_vazio(*m)

In [ ]:
def bits(n):
    bits = [] 
    while n > 0:
        r = n%2
        n = n//2
        bits.append(r)
    return bits


In [ ]:
bits(16)

In [ ]:
def soma_bits(a,b, n ):
    r = a+b
    for i in range(n-1):
        if r[i] == 2:
            r[i] = 0
            r[i+1] +=1
    return r